# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,iqaluit,63.7506,-68.5145,-9.15,79,100,4.63,CA,1711312572
1,1,mcpherson,38.3708,-97.6642,12.83,86,100,13.67,US,1711312572
2,2,olonkinbyen,70.9221,-8.7187,-4.96,66,100,6.47,SJ,1711312572
3,3,puerto ayora,-0.7393,-90.3518,30.98,93,49,4.47,EC,1711312572
4,4,varberg,57.1056,12.2508,4.57,93,100,2.16,SE,1711312321


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
%%capture --no-display

# Configure the map plot
map_city_data= city_data_df.hvplot.points(
    "Lng",
    "Lat",
     size = "Humidity",
     color= "City"
)

# Display the map
map_city_data

:Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [12]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city_data_df = city_data_df.loc[(city_data_df["Max Temp"] < 30) & 
                                      (city_data_df["Max Temp"] > 20) & 
                                      (city_data_df["Wind Speed"] < 6) & 
                                      (city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
ideal_city_data_df = ideal_city_data_df.dropna()

# Display sample data
ideal_city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
72,72,veraval,20.9000,70.3667,24.32,76,0,4.92,IN,1711312580
85,85,remire-montjoly,4.9167,-52.2667,29.02,65,0,5.14,GF,1711312581
177,177,kwang binh,17.4833,106.6000,23.42,91,0,2.52,VN,1711312591
190,190,brisas de zicatela,15.8369,-97.0419,29.29,70,0,3.60,MX,1711312592
198,198,malvan,16.0667,73.4667,25.66,76,0,5.73,IN,1711312593


### Step 3: Create a new DataFrame called `hotel_df`.

In [13]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
72,veraval,IN,20.9000,70.3667,76,
85,remire-montjoly,GF,4.9167,-52.2667,65,
177,kwang binh,VN,17.4833,106.6000,91,
190,brisas de zicatela,MX,15.8369,-97.0419,70,
198,malvan,IN,16.0667,73.4667,76,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [14]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accomodation.hotel",
    "key": geoapify_key,
    "limit": 1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude= row["Lat"]
    longitude= row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude}, {latitude}, {radius}"
    params["bias"] = f"proximity:{longitude}, {latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
veraval - nearest hotel: No hotel found
remire-montjoly - nearest hotel: No hotel found
kwang binh - nearest hotel: No hotel found
brisas de zicatela - nearest hotel: No hotel found
malvan - nearest hotel: No hotel found
sur - nearest hotel: No hotel found
mangrol - nearest hotel: No hotel found
galliano - nearest hotel: No hotel found
nakhon thai - nearest hotel: No hotel found
merowe - nearest hotel: No hotel found
ad dabbah - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
72,veraval,IN,20.9000,70.3667,76,No hotel found
85,remire-montjoly,GF,4.9167,-52.2667,65,No hotel found
177,kwang binh,VN,17.4833,106.6000,91,No hotel found
190,brisas de zicatela,MX,15.8369,-97.0419,70,No hotel found
198,malvan,IN,16.0667,73.4667,76,No hotel found
287,sur,OM,22.5667,59.5289,51,No hotel found
326,mangrol,IN,21.1167,70.1167,79,No hotel found
327,galliano,US,29.4422,-90.2992,60,No hotel found
445,nakhon thai,TH,17.1006,100.8374,42,No hotel found
474,merowe,SD,18.4833,31.8167,15,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [20]:
%%capture --no-display

# Configure the map plot
hotel_data= hotel_df.hvplot.points(
    "Lng",
    "Lat",
    tiles="OSM",
     color= "Hotel Name"
)

# Display the map
hotel_data

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Hotel Name)